Fine tuned model(Biogpt) deployment on streamlit using Ngrok

Install libraries

In [ ]:
!pip install transformers==4.29.2
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=101323239ceeab0dacc9db49d3dab9aee3f5d552c5e7f127c1d236f9669b1e84
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [ ]:
!pip install streamlit -q




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 15.1 MB/s eta 0:00:00


## Streamlit file

In [ ]:
#for the cpu code.
#device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#generated = generated.to(device)
#model.to(device)
#model.eval();
%%writefile app.py
import streamlit as st

st.title("HEALTHQUERY")
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler

from IPython.display import clear_output
from transformers import BioGptTokenizer, BioGptForCausalLM, TrainerCallback
from transformers import pipeline
#summarizer_bart = pipeline("summarization", model="facebook/bart-large-cnn")
#summarizer_knnkar = pipeline("summarization", model="knkarthick/MEETING_SUMMARY")
summarizer_sshle = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

import os
DEBUG           = False

INPUT_DIR       = 'articles'

USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'

MODEL           = 'gpt2' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}

MAXLEN          = 256  #{768, 1024, 1280, 1600}

TRAIN_SIZE      = 0.8

if USE_APEX:
    TRAIN_BATCHSIZE = 16
    BATCH_UPDATE    = 128
else:
    TRAIN_BATCHSIZE = 8
    BATCH_UPDATE    = 256

EPOCHS          = 3
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2020


DEVIDE_BY = 20

os.environ['WANDB_DISABLED'] = 'true'



tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
model = BioGptForCausalLM.from_pretrained('/content/drive/MyDrive/All models/biogpt')



input_text = st.text_input("Please Provide your text:")
title = input_text
prompt = SPECIAL_TOKENS['bos_token'] + title + SPECIAL_TOKENS['sep_token']
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)
device = torch.device("cuda")
model.cuda()
model.eval();
from heapq import nsmallest

# Generate text

if len(input_text)>0:
  sample_outputs = model.generate(generated,
                                do_sample=True,
                                max_length=MAXLEN,
                                top_k=10,
                                top_p=0.7,
                                temperature=0.5,
                                repetition_penalty=2.0,
                                num_return_sequences=1
                                )


  # Initialize an empty list to store the perplexity and text pairs
  perplexity_text_pairs = []


  for i, sample_output in enumerate(sample_outputs):
      text = tokenizer.decode(sample_output,skip_special_tokens=True)
      a = len(title)+25
      st.write(a)
      st.write(("{}: {}\n\n".format(i+1,  text[a:])))
      # all questions print in above cod
  bart_Val=text[a:]
  #x=summarizer(bart_Val, max_length=200, min_length=30, do_sample=False)
  #st.write('-------Bart summarization-----')
  #st.write(x[0]['summary_text'])

  #summary=summarizer_knnkar(bart_Val, max_length=200, min_length=30, do_sample=False)
  #st.write('-------MEETING_SUMMARY-----')
  #st.write(summary[0]['summary_text'])

  distl=summarizer_sshle(bart_Val, max_length=200, min_length=30, do_sample=False)
  st.write('-------distilbart_cnn_12-6 model -----')
  st.write(distl[0]['summary_text'])



else:
  st.write('Welcome to GPT2')
# Create a "Regenerate" button


# Display output



Writing app.py


## Ngrok Configuration

In [ ]:

!pip install pyngrok==5.2.1


from pyngrok import ngrok





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19771 sha256=511d3c3ad86e79dfcf70c7e0ea5fbec94c3baa6dfea054f51a277987c9a71eb2
  Stored in directory: /root/.cache/pip/wheels/6c/e1/46/8d60711cb43fb2e055fb69bb9964f91c9a5046f7924d2996ac
Successfully built pyngrok


In [ ]:
ngrok.set_auth_token("2LoeoedCqusM7VxjDHJfm1b9TJg_5wRMcFESTSHY6u26tUoMj") #ngrok.com





In [ ]:
!killall ngrok

ngrok: no process found


In [ ]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://243a-34-145-215-26.ngrok-free.app" -> "http://localhost:80"
